In [5]:
pip install faiss

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [ ]:
import os
import warnings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ[' LANGCHAIN_PROJECT'] = "book-chapter-6" 
warnings.filterwarnings('ignore')

llm = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-1106-preview")

#you are a music fan chatbot

memory = ConversationBufferMemory(return_messages=True)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an events chatbot who likes live music, ask your user for their name and some examples of bands they want to go and see"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

chain_with_memory = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | llm
)

inputs = {"input": "hi"}
response = chain_with_memory.invoke(inputs)
response

memory.save_context(inputs, {"output": response.content})

inputs = {"input": "My name is Adrian and I like the high flying birds"}
response = chain_with_memory.invoke(inputs)
response

memory.save_context(inputs, {"output": response.content})

inputs = {"input": "I'm looking for gigs in London this summer"}
response = chain_with_memory.invoke(inputs)
response

memory.save_context(inputs, {"output": response.content})

memory.load_memory_variables({})


In [13]:

# Understanding an example of using memory in Langchain


import os
import warnings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''

warnings.filterwarnings('ignore')

llm = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-1106-preview")
memory = ConversationBufferMemory(return_messages=True, memory_key="conversation_history")
memory.load_memory_variables({})
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an events chatbot who likes live music, ask your user for their name and some examples of bands they want to go and see"),
        MessagesPlaceholder(variable_name="conversation_history"),
        ("human", "{input}"),
    ]
)

chain_with_memory = (
    RunnablePassthrough.assign(
        conversation_history=RunnableLambda(memory.load_memory_variables) | itemgetter("conversation_history")
    )
    | prompt
    | llm
    | 
)

inputs = {"input": "hi"}
response = chain_with_memory.invoke(inputs)
memory.save_context(inputs, {"output": response.content})

inputs = {"input": "My name is Adrian and I like the high flying birds"}
response = chain_with_memory.invoke(inputs)
memory.save_context(inputs, {"output": response.content})

inputs = {"input": "I'm looking for gigs in London this summer"}
response = chain_with_memory.invoke(inputs)
memory.save_context(inputs, {"output": response.content})

memory.load_memory_variables({})



{'conversation_history': [HumanMessage(content='hi'),
  AIMessage(content="Hello there! I'm thrilled to chat with a fellow live music enthusiast. What's your name, and could you share with me some examples of bands or artists you're eager to go see live?"),
  HumanMessage(content='My name is Adrian and I like the high flying birds'),
  AIMessage(content="Nice to meet you, Adrian! The High Flying Birds, as in Noel Gallagher's High Flying Birds, have some great tunes. If you're into their sound, you might also enjoy other bands from that Britpop era or similar genres. Are there any other bands or artists you're interested in seeing live?"),
  HumanMessage(content="I'm looking for gigs in London this summer"),
  AIMessage(content="That's fantastic, Adrian! London's music scene is always buzzing, especially in the summer. While I can't provide real-time listings, I can suggest that you check out some popular venues for gigs like the O2 Academy Brixton, the Roundhouse, or the Royal Albert H

In [31]:
summary_memory = ConversationSummaryMemory(llm=llm, return_messages=True)
summary_memory.save_context({"input": "hi I'm adrian tell me about the season of winter"}, {"output": "whats up"})
summary_memory.load_memory_variables({})


from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)
memory.buffer

'Adrian greeted the AI, and the AI responded casually.'

In [30]:
# Using ConversationSummaryBufferMemory

import os
import warnings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
llm = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-1106-preview")
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=150, return_messages=True)
conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True,
)

response = conversation_with_summary.predict(input = "My name is Adrian and I like 1990s music, can you remember my 3 favourite bands, Nirvana, Offspring and Underworld")

response = conversation_with_summary.predict(input = "What's my second favourite band")

response

memory.buffer

memory.load_memory_variables({})




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: My name is Adrian and I like 1990s music, can you remember my 3 favourite bands, Nirvana, Offspring and Underworld
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='My name is Adrian and I like 1990s music, can you remember my 3 favourite bands, Nirvana, Offspring and Underworld'), AIMessage(content="Hello Adrian! It's great to hear that you're a

{'history': [SystemMessage(content='Adrian, who likes 1990s music, asked the AI to remember his three favorite bands: Nirvana, Offspring, and Underworld. The AI acknowledged the iconic status of these bands in their respective genres and assured Adrian it would remember his preferences.'),
  HumanMessage(content="What's my second favourite band"),
  AIMessage(content="Your second favorite band is Offspring, Adrian. They really made a mark in the '90s with their energetic punk rock sound. If you'd like to discuss some of their albums or hits, or anything else related to the band, just let me know!")]}

In [ ]:
#Debugging and monitoring
from langchain.globals import set_verbose, set_debug
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.output_parsers import PydanticOutputParser

# Enable verbose logging
set_verbose(False)

# Enable debugging
set_debug(True)

llm = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-1106-preview")

# Initialize the LLM (Language Model) with OpenAI's GPT
# Make a simple request to the LLM
# Define the prompt template
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)


# Define the chain
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
)

# Invoke the chain with a topic
response = chain.invoke({"topic": "cows."})
response

In [ ]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.schema.runnable import RunnableLambda

def function_1(x: int) -> int:
    return x + 1

def function_1(x: int) -> int:
    return x * 2
#create chain
runnable = RunnableLambda(fun_1) | RunnableLambda(fun_2)

runnable.invoke(1, config={'callbacks': [ConsoleCallbackHandler()]})


In [ ]:
pip install -U langchain-community faiss-cpu langchain-openai tiktoken

In [2]:
import os
from datetime import datetime
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import chromadb
from langchain.vectorstores import Chroma
from langchain.docstore import InMemoryDocstore
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter
os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ[' LANGCHAIN_PROJECT'] = "book-chapter-6" 

llm = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-1106-preview")
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

chroma_client = chromadb.PersistentClient(path="./embeddings")

history_vectorstore = Chroma(collection_name="conversation_history", client=chroma_client, embedding_function=embeddings)

retriever = history_vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever, memory_key="conversation_history", input_key="input")

# using the memory object to save information from conversations
memory.save_context({"input": "Hi, my name is Adrian, how are you?"}, {"output": "nice to meet you Adrian"}) 
memory.save_context({"input": "My favourite sport is running"}, {"output": "that's a good thing to be doing"})

_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI assistant. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{conversation_history}

(Don't use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["conversation_history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_history = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True
)

# the history will be used in the prompt here
conversation_with_history.invoke({"input":"what gear do i need to buy to get started in my sport"})




similar_documents = history_vectorstore.similarity_search(query="do i need a torch", k=1)

print(history_vectorstore._collection.get())  #without embeddings
print(history_vectorstore._collection.get(include=['embeddings'])) # with embeddings

document_count = history_vectorstore._collection.count()
print(document_count)
for document in similar_documents:
    print(document)


C:\Users\AdrianThomspon\anaconda3\envs\python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI assistant. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: what gear do i need to buy to get started in my sport
response: Hello Adrian! If you're looking to get started in running, the good news is that you don't need a lot of gear to begin. Here are some essentials to consider:

1. **Running Shoes**: The most important piece of equipment for any runner. Look for a pair that fits well and is designed for the type of running you plan to do (road, trail, racing, etc.). It's often worth visiting a specialty running store where you can get advice and a proper fitting.

2. **Technical Clothing**: Invest in moisture-wicking fabrics to keep you dry and comfortable. This includes shorts, shirts, s

In [3]:
print(history_vectorstore._collection.get(include=['embeddings'])) # with embeddings

{'ids': ['4462820c-046a-4ecf-9efe-1b8fc508a27e', '7af3ed94-b790-11ee-916e-bc6ee202f7d7', '7b3ebc38-b790-11ee-b23d-bc6ee202f7d7', '874a5f15-b790-11ee-8819-bc6ee202f7d7', 'cbce0a03-8994-43e0-af92-ddbf03bb5789', 'f0f91c1c-3156-4a9b-8cff-f0f634f18325'], 'embeddings': [[0.004683004839289084, 0.0012212008338419516, 0.01545199460474712, -0.0384377960044765, -0.0197954648677996, 0.02990460564911056, -0.007642715183787835, -0.02534332140241447, -0.020589847579649583, -0.025138318691235752, 0.01915483605197867, 0.006339033168719843, -0.007565839632757073, -0.013017600448015037, -0.0006077976600309935, 0.006252548232017892, 0.03323588294195948, -0.016246375453952085, 0.0024808394431992844, -0.03149336734752049, 0.007072554885781451, -0.00663692645283296, 0.02736771106811163, -0.013261039770555995, -0.015272618163788255, -0.01467042595932957, 0.018808896305170866, -0.004023156320803268, -0.021038286819401723, -0.0075914649716544115, 0.007143023869257251, 0.022793613686305614, -0.0060379370664905, 

In [8]:
import chromadb
import os
from langchain.vectorstores import Chroma




chroma_client = chromadb.PersistentClient(path="./embeddings")

os.environ['OPENAI_API_KEY'] = ''


embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


history_vectorstore = Chroma(collection_name="conversation_history", client=chroma_client, embedding_function=embeddings)
similar_documents = history_vectorstore.similarity_search(query="do i need a torch", k=1)

#print(history_vectorstore._collection.get())
#print(history_vectorstore._collection.get(include=['embeddings']))

#print(len(embeddings.embed_documents(["EMBED THIS"])[0]))


count = history_vectorstore._collection.count()
print(count)
for document in similar_documents:
    print(document)


3
page_content="input: what gear do i need to buy to get started in my sport\nresponse: Hello Adrian! If you're looking to get started in running, the good news is that you don't need a lot of gear to begin. Here are some essentials to consider:\n\n1. **Running Shoes**: The most important piece of equipment for any runner. Look for a pair that fits well and is designed for the type of running you plan to do (road, trail, racing, etc.). It's often worth visiting a specialty running store where you can get advice and a proper fitting.\n\n2. **Technical Clothing**: Invest in moisture-wicking fabrics to keep you dry and comfortable. This includes shorts, shirts, socks, and for women, a supportive sports bra. Avoid cotton as it retains moisture and can lead to chafing.\n\n3. **Running Watch or App**: While not essential, a watch or smartphone app can help you track your distance, pace, and time. This can be motivating and informative as you progress in your running journey.\n\n4. **Hydratio

In [20]:
#Document Loader

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(["https://en.wikipedia.org/wiki/New_York_City#Economy"])

docs = loader.load()
print(len(docs))


#Text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20,)
documents = text_splitter.split_documents(docs)

print(len(documents))
print(documents[0])
print(documents[3])

1
348
page_content='New York City - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\nLanguages\n\nLanguage links are at the top of the page.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1Etymology\n\n\n\n\n\n\n\n2History\n\n\n\nToggle History subsection\n\n\n\n\n\n2.1Early history\n\n\n\n\n\n\

In [3]:
# creating a Tavaly search tool
import os

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor



os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = "book-chapter-6" 

os.environ["TAVILY_API_KEY"] = ""



from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults


In [ ]:
pip install -U langchain-community tavily-python

In [62]:
#Simple OpenAI function example

from openai import OpenAI
from tavily import TavilyClient
import json

client = OpenAI()

tavily = TavilyClient(api_key="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")

messages = [{"role": "user", "content": "explain how to play chess"}]

tools = [{ 
    "type":"function",
     "function": {
                 "name": "get_news_articles", 
        "description": "Retrieve news articles based on a specific topic",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic or subject of interest for the news articles"
                }
            }
        }
     }
}] 

def get_news_articles(topic): 
    response = tavily.search(query=topic)
    return response['results'] 


messages = [{"role": "user", "content": "how can I create a list of examples of test user complaints for a sports brand"}]

tools = [{ 
    "type":"function",
     "function": {
        "name": "get_sports_complaints", 
        "description": "Get a list of complaints specific to a sports brand",
        "parameters": {
            "type": "object",
            "properties": {
                "complaints": {
                    "type": "array",
                    "items":{
                        "type":"string",
                        "description": "A customer complaint"
                    },
                    "description": "List of customer complaint strings"
                }
            }
        }
     }
}] 



response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "get_sports_complaints"}}
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(response_message)
print(tool_calls)



ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_t4ZGrHDW4NxHtFfRXeRRngwA', function=Function(arguments='{}', name='get_sports_complaints'), type='function')])
[ChatCompletionMessageToolCall(id='call_t4ZGrHDW4NxHtFfRXeRRngwA', function=Function(arguments='{}', name='get_sports_complaints'), type='function')]


In [6]:
import os
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_openai import ChatOpenAI

os.environ["TAVILY_API_KEY"] = "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"

# create the llM
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

#create the tool
tavily_search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=tavily_search)
# print(tavily_tool.name)
print(tavily_tool.description)
# print(tavily_tool.args)
#test the results we get back
tavily_tool.invoke({"query": "how many named storms have there been this year in the UK"})

#Create our tools array

prompt_template = hub.pull("langchain-ai/openai-functions-template")

instructions = "you are a helpful agent"
openai_prompt = f"{prompt_template}\nInstructions: {instructions}"

print(openai_prompt)

tools = [tavily_tool]

openai_agent = create_openai_functions_agent(llm, tools, openai_prompt) 

openai_agent_executor = AgentExecutor(agent=openai_agent, tools=tools)

openai_agent_executor.invoke({"input": "how many named storms have there been this year in the UK"})

# Create the OpenAI Functions Agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


[{'url': 'https://www.bbc.co.uk/news/uk-68070561',
  'content': 'In a typical UK autumn/winter, there would be six or seven named storms - getting to the letters F or G. The current storm season has brought the most named storms since 2015/16, the season the ...'},
 {'url': 'https://news.sky.com/story/uk-weather-why-have-there-been-so-many-storms-this-year-and-could-more-be-on-the-way-13055306',
  'content': "This storm season is only the second time the letter J has been reached in the alphabet of named storms. What's behind the stormy weather - and are more on the way? Wednesday 24 January 2024 14:21, UK"},
 {'url': 'https://www.metoffice.gov.uk/weather/warnings-and-advice/uk-storm-centre/uk-storm-season-2022-23',
  'content': 'The 2022/23 storm season saw only two named storms across the year, with these two storms coming late in the season with Storms Antoni and Betty impacting the UK and Ireland in August 2023. However, If a storm is named by another national meteorological servic

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


tools = [tavily_tool]

# Create out prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
tavily_tool.description
open_ai_agent = create_openai_functions_agent(llm, tools,prompt )
open_ai_agent_executor = AgentExecutor(agent=open_ai_agent, tools=tools)

# Invoke the agent executor
result = open_ai_agent_executor.invoke({"input": "how many named storms have there been in 2024 in the UK"})
print(result)

{'input': 'how many named storms have there been in 2024 in the UK', 'output': 'The UK has seen an impressive amount of storms in 2024 so far, and the storm names for the year include Agnes, Babet, Ciarán, Henk, Isha, and Jocelyn. As of now, there have been at least six named storms in the UK in 2024.'}


In [ ]:
pip install openmeteo-requests
pip install requests-cache retry-requests numpy pandas

In [104]:
pip install requests-cache retry-requests numpy pandas

   ---------------------------------------- 0.0/60.3 kB ? eta -:--:--
   ---------------------------------------- 60.3/60.3 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
   ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [106]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 51.5085,
	"longitude": -0.1257,
	"current": "temperature_2m",
	"hourly": "temperature_2m",
	"timezone": "GMT"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()

print(f"Current time {current.Time()}")
print(f"Current temperature_2m {current_temperature_2m}")

Coordinates 51.5°E -0.12000012397766113°N
Elevation 23.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Current time 1707168600
Current temperature_2m 9.899999618530273


In [15]:
import requests
import os
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

from langchain.tools import BaseTool
from typing import Union
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain import hub
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = "book-chapter-6" 

desc = (
    "A weather tool optimized for comprehensive up to date weather information. Useful for when you need to answer questions about the weather, use this tool to answer questions about the weather for a specific location. To use the tool, you must provide at the following parameters" 
    "['latitude', 'longitude']."
)

llm = ChatOpenAI(model="gpt-3.5-turbo-1106")


class GetWeatherByLocationTool(BaseTool):
    name = "Weather reporter"
    description = desc

    def _run(
        self,
        latitude: Union[int, float],
        longitude: Union[int, float]) -> None:
        url = "https://api.open-meteo.com/v1/forecast"
        # Parameters for the API request
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true"  # Get current weather; adjust parameters as needed
            }
    
        # Make the GET request to Open-Meteo
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Extract and print the current weather details
            current_weather = data.get("current_weather")
            temperature = current_weather.get("temperature")
            current_rain = current_weather.get("current_rain")
            result_string = f"Current Temperature: {temperature}°C, Rain: {current_rain}"
            return result_string
        else:
            print("Failed to retrieve weather data")
            return "no weather data"
    

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")


tools = [GetWeatherByLocationTool()]

prompt = prompt = hub.pull("hwchase17/structured-chat-agent")

agent = create_structured_chat_agent(llm,tools,prompt)

executor = AgentExecutor(agent=agent, tools=tools,handle_parsing_errors=True)

output = executor.invoke({"input":"whats the weather like in waddesdon"})

print(output)

output = executor.invoke({"input":"whats the latest information about the kings cancer diagnosis"})

print(output)

{'input': 'whats the weather like in waddesdon', 'output': 'The current temperature in Waddesdon is 5.5°C and there is no rain at the moment.'}
{'input': 'whats the latest information about the kings cancer diagnosis', 'output': "I don't have the latest information about the king's cancer diagnosis."}
